# Custom Header Routing with Seldon and Ambassador

This notebook shows how you can deploy Seldon Deployments with custom Ambassador configuration.

## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core) running this notebook
 - A running Kubernetes cluster with kubectl authenticated
 - [Helm client](https://helm.sh/)

### Creating a Kubernetes Cluster

Follow the [Kubernetes documentation to create a cluster](https://kubernetes.io/docs/setup/).

Once created ensure ```kubectl``` is authenticated against the running cluster.

# Setup

In [ ]:
!kubectl create namespace seldon

In [ ]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

In [ ]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

# Install Helm

In [ ]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

In [ ]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

## Start seldon-core

In [ ]:
!helm install ../../../helm-charts/seldon-core-crd --name seldon-core-crd --set usage_metrics.enabled=true

In [ ]:
!helm install ../../../helm-charts/seldon-core --name seldon-core --namespace seldon  --set ambassador.enabled=true

In [ ]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-seldon-apiserver
!kubectl rollout status deploy/seldon-core-ambassador

## Set up REST and gRPC methods

**Ensure you port forward ambassador**:

```
kubectl port-forward $(kubectl get pods -n seldon -l service=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

## Launch main model

We will create a very simple Seldon Deployment with a dummy model image `seldonio/mock_classifier:1.0`. This deployment is named `example`. We will add custom Ambassador config which sets the Ambassador prefix to `/mycompany/ml`

We must ensure we set the correct service endpoint. Seldon Core creates an endpoint of the form:
 
`<spec.name>-<metadata.name>.<namespace>:<port>`

Where

  * `<spec-name>` is the name you give to the Seldon Deployment spec: `production-model` below
  * `<metadata.name>` is the metadata name in the Seldon Deployment: `example` below
  * `<namespace>` is the namespace your Seldon Deployment is deployed to
  * `<port>` is the port either 8000 for REST or 5000 for gRPC
  
This will allow you to set the `service` value in the Ambassador config you create. So for the example below we have:

```
service: production-model-example.seldon:8000
```
  
  

In [1]:
!pygmentize model_custom_ambassador.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "example"
    },
    "spec": {	
        "name": "production-model",
	"annotations": {
	    "seldon.io/ambassador-config":"apiVersion: ambassador/v0\nkind: Mapping\nname: seldon_example_rest_mapping\nprefix: /mycompany/ml/\nservice: production-model-example.seldon:8000\ntimeout_ms: 3000"
	},	
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mock_classifier:1.0",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier"
                            }
                        ],
                        "terminationGracePeriodSeconds": 1
                    }}
                 

In [ ]:
!kubectl create -f model_custom_ambassador.json

In [ ]:
!kubectl rollout status deploy/production-model-single-7cd068f

### Get predictions

In [ ]:
import sys
sys.path.append("../../../notebooks")
from seldon_utils import *
API_AMBASSADOR="localhost:8003"

#### REST Request

In [ ]:
r = rest_request_ambassador("example",None,API_AMBASSADOR,prefix="/mycompany/ml")
print(r.text)